In [1]:
!pip install transformers datasets torch scikit-learn pandas
!pip install pandas
!pip install --upgrade transformers torch accelerate datasets scikit-learn pandas
!pip list | grep accelerate
!pip install accelerate

accelerate                               1.6.0


In [1]:
import pandas as pd

# Load the dataset
df = pd.read_csv("behavioral_economics_dataset.csv")

# Check dataset structure
print(df.head())

  Student_ID                 Concept  \
0      S1000        Endowment Effect   
1      S1001        Endowment Effect   
2      S1002          Anchoring Bias   
3      S1003  Hyperbolic Discounting   
4      S1004  Hyperbolic Discounting   

                                    Student_Response Faculty_Grade  
0     People fear losses more than they value gains.            B+  
1       Choice architecture can influence decisions.             A  
2  Past investments affect future decisions irrat...             B  
3          Risk perception changes based on framing.            C+  
4       Choice architecture can influence decisions.            C+  


In [2]:
# Rename columns if needed (ensure they match expected format)
df = df.rename(columns={"Student_Response": "text", "Faculty_Grade": "label"})

# Check for missing values
df = df.dropna()


In [3]:
# Convert Grades to Numeric Labels for Classification
grade_mapping = {'A+': 10, 'A': 9, 'A-': 8, 'B+': 7, 'B': 6, 'B-': 5, 'C+': 4, 'C': 3, 'C-': 2, 'D': 1, 'F': 0}
df['label'] = df['label'].map(grade_mapping)

print(df.head()) 

  Student_ID                 Concept  \
0      S1000        Endowment Effect   
1      S1001        Endowment Effect   
2      S1002          Anchoring Bias   
3      S1003  Hyperbolic Discounting   
4      S1004  Hyperbolic Discounting   

                                                text  label  
0     People fear losses more than they value gains.      7  
1       Choice architecture can influence decisions.      9  
2  Past investments affect future decisions irrat...      6  
3          Risk perception changes based on framing.      4  
4       Choice architecture can influence decisions.      4  


In [4]:
from transformers import AutoTokenizer

# Choose a Pretrained Model (e.g., DistilBERT)
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)


/home/ashok/langchain/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Tokenizing Function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


In [6]:
from datasets import Dataset

# Convert DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

In [7]:
# Tokenize Data
tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map: 100%|███████████████████████████| 100/100 [00:00<00:00, 4606.14 examples/s]


In [8]:
# Split into Train-Test
train_test_split = tokenized_datasets.train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]

In [9]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch


In [10]:
# Load Pretrained Model for Classification
num_labels = len(grade_mapping)  # Number of grading categories
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
import torch
torch.cuda.empty_cache()

In [11]:
# Define Training Arguments (Use eval_strategy instead of evaluation_strategy)
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",   # <- Fixed the deprecated warning
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    fp16=True,
    gradient_accumulation_steps=4,
    weight_decay=0.01,
)

In [12]:
# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer
)

/tmp/ipykernel_5407/2894724606.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [13]:
# Train Model
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,2.432422
2,No log,2.415137
3,No log,2.409082


TrainOutput(global_step=15, training_loss=2.394148763020833, metrics={'train_runtime': 112.6717, 'train_samples_per_second': 2.13, 'train_steps_per_second': 0.133, 'total_flos': 31797278392320.0, 'train_loss': 2.394148763020833, 'epoch': 3.0})

In [14]:
!pip install numpy

import numpy as np
from sklearn.metrics import accuracy_score, classification_report

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
# Function to Compute Metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}


In [17]:
# Evaluate Model
trainer.evaluate()

model.save_pretrained("grading_model")
tokenizer.save_pretrained("grading_model")

model.save_pretrained("./grading_model")
tokenizer.save_pretrained("./grading_model")


('./grading_model/tokenizer_config.json',
 './grading_model/special_tokens_map.json',
 './grading_model/vocab.txt',
 './grading_model/added_tokens.json',
 './grading_model/tokenizer.json')

In [18]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Load trained model and tokenizer
model_path = "./grading_model"  # Use "your-hf-username/grading_model" if using Hugging Face Hub
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)


In [22]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Correct grade_mapping used during training
train_grade_mapping = {
    'A+': 10, 'A': 9, 'A-': 8, 'B+': 7, 'B': 6, 'B-': 5, 
    'C+': 4, 'C': 3, 'C-': 2, 'D': 1, 'F': 0
}

# Modify grade mapping for prediction
grade_mapping_for_prediction = {
    10: "A+", 9: "A", 8: "A-", 7: "B+", 6: "B", 5: "B-", 
    4: "C+", 3: "C", 2: "C-", 1: "D", 0: "F"
}

# Function to predict grade
def predict_grade(student_response):
    inputs = tokenizer(student_response, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    
    predicted_class = torch.argmax(outputs.logits, dim=1).item()
    
    # Ensure the predicted class is within the valid grade range
    if predicted_class in grade_mapping_for_prediction:
        return grade_mapping_for_prediction[predicted_class]
    else:
        # Handle cases where the predicted class is out of bounds (for safety)
        return "Unknown Grade"

# Example: Predicting grade for a student answer
student_response = "The Industrial Revolution was a period of major industrialization..."
predicted_grade = predict_grade(student_response)
print(f"Predicted Grade: {predicted_grade}")


Predicted Grade: A+
